In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import json
import requests
import os


In [9]:
# Define the parameters X and Y
X = 0.5 # The minimum interval in minutes between creating PDFs
Y = 20 # The number of characters to use for naming the PDFs

In [10]:
# Launch Microsoft Edge with remote debugging enabled using the command: 

#!msedge.exe --remote-debugging-port=9222
#!C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe --remote-debugging-port=9222
#!C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe
#%run C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe --remote-debugging-port=9222
#%run "C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe" --remote-debugging-port=9222
#%run C:/PROGRA~2/MICROS~1/Edge/Application/msedge.exe --remote-debugging-port=9222
#%run 'C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe' --remote-debugging-port=9222
#%run C:/Program\ Files\ (x86)/Microsoft/Edge/Application/msedge.exe --remote-debugging-port=9222

# At last
import subprocess
subprocess.run(["C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe", "--remote-debugging-port=49152"])


CompletedProcess(args=['C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe', '--remote-debugging-port=49152'], returncode=0)

In [12]:
# Create a directory called sidebars if it does not exist
if not os.path.exists("sidebars"):
    os.mkdir("sidebars")

# Create a dictionary to store the last modified time of each sidebar PDF
last_modified = {}

# Create an infinite loop to run the script in the background
while True:

    # Get the session IDs and URLs of the existing browser sessions using the command: curl http://localhost:9222/json
    response = requests.get("http://localhost:49200/json")
    data = json.loads(response.text)

    # Loop through each browser session
    for session in data:

        # Get the session ID and URL of the current browser session
        session_id = session["id"]
        URL = session["webSocketDebuggerUrl"]

        # Create a webdriver object with the session ID and URL using the command: driver = webdriver.Remote(command_executor=URL, desired_capabilities={})
        driver = webdriver.Remote(command_executor=URL, desired_capabilities={})

        # Attach the webdriver object to the existing browser session using the command: driver.session_id = session_id
        driver.session_id = session_id

        # Get the sidebar element
        sidebar = driver.find_element_by_id("h_container")

        # Get the HTML content of the sidebar element
        sidebar_html = sidebar.get_attribute("innerHTML")

        # Get the title of the current web page
        title = driver.title

        # Generate a name for the PDF file based on the first Y characters of the title
        pdf_name = title[:Y] + ".pdf"

        # Generate a path for the PDF file based on the sidebars directory and the PDF name
        pdf_path = os.path.join("sidebars", pdf_name)

        # Check if there is a previous PDF file with the same name
        if pdf_name in last_modified:

            # Get the last modified time of the previous PDF file
            previous_time = last_modified[pdf_name]

            # Get the current time
            current_time = time.time()

            # Calculate the time difference in minutes
            time_diff = (current_time - previous_time) / 60

            # Check if the time difference is greater than or equal to X
            if time_diff >= X:

                # Save the sidebar contents as a PDF file using a JavaScript executor, overwriting the previous PDF file
                driver.execute_script(f"window.sidebarPDF = document.createElement('a');")
                driver.execute_script(f"window.sidebarPDF.href = URL.createObjectURL(new Blob([{sidebar_html}], {{type: 'application/pdf'}}));")
                driver.execute_script(f"window.sidebarPDF.download = '{pdf_path}';")
                driver.execute_script(f"window.sidebarPDF.click();")

                # Update the last modified time of the PDF file in the dictionary
                last_modified[pdf_name] = current_time

        else:

            # Save the sidebar contents as a PDF file using a JavaScript executor, creating a new PDF file
            driver.execute_script(f"window.sidebarPDF = document.createElement('a');")
            driver.execute_script(f"window.sidebarPDF.href = URL.createObjectURL(new Blob([{sidebar_html}], {{type: 'application/pdf'}}));")
            driver.execute_script(f"window.sidebarPDF.download = '{pdf_path}';")
            driver.execute_script(f"window.sidebarPDF.click();")

            # Get the current time
            current_time = time.time()

            # Add the PDF name and the last modified time to the dictionary
            last_modified[pdf_name] = current_time

    # Close all browser sessions
    for driver in webdriver.Remote.get_sessions():
        driver.quit()

    # Wait for X minutes before repeating the loop
    time.sleep(X * 60)


ConnectionError: HTTPConnectionPool(host='localhost', port=49200): Max retries exceeded with url: /json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001618F392F70>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))